In [1]:
import os
import sys

In [2]:
os.chdir('/home/au643300/DataHandling/')
sys.path.append('/home/au643300/DataHandling/')
os.environ['HDF5_USE_FILE_LOCKING']="FALSE"

In [3]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, as_completed,wait
import glob
from src.data.tonetCDF import to_netcdf
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from src.interim import read_valdata
import numpy as np
import importlib
import dask

In [4]:
sns.set_theme()
dask.config.set({"distributed.comm.timeouts.tcp": "50s"})

In [5]:
batches=10

In [6]:
cluster=SLURMCluster(cores=6,
                     processes=1,
                     memory="22GiB",
                     queue='q64',
                     walltime='0-00:45:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     extra=['--resources mem=15',"--lifetime", "40m","--lifetime-stagger", "4m"]
                    )
                    

In [7]:
client=Client(cluster)

In [8]:
cluster.adapt(minimum_jobs=1,maximum_jobs=8)

In [9]:
client

Client Scheduler: tcp://10.100.8.23:40507 Dashboard: http://10.100.8.23:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [10]:
files=glob.glob("/home/au643300/NOBACKUP/interim/*nc")
ds=xr.open_mfdataset(files,parallel=True,combine='nested',concat_dim='time',chunks='auto')
#ds=xr.open_mfdataset("/home/au643300/NOBACKUP/interim/*nc",parallel=True,combine='nested',concat_dim='time',chunks='auto')

In [11]:
ds

<xarray.Dataset>
Dimensions:  (time: 1, x: 256, y: 201, z: 256)
Coordinates:
  * x        (x) float64 0.0 0.04688 0.09375 0.1406 ... 11.81 11.86 11.91 11.95
  * y        (y) float64 2.0 2.0 2.0 1.999 ... 0.00111 0.0004934 0.0001234 0.0
  * z        (z) float64 -3.0 -2.977 -2.953 -2.93 ... 2.906 2.93 2.953 2.977
  * time     (time) float64 2.632e+03
Data variables:
    u_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    v_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    w_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr1      (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.71   (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.2    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.025  (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>

Constants from the simulation

In [12]:
Re_Tau = 395 #Direct from simulation
Re = 10400 #Direct from simulation
nu = 1/Re #Kinematic viscosity
u_tau = Re_Tau*nu #The friction velocity 
#y_plus=(y*utau/nu)


changes y til y+. goes from 790 to 0. so y_plus[0]=790

In [13]:
ds=ds.assign_coords(y=(ds.y*u_tau/nu))
ds=ds.rename({'y':'y_plus'})
ds

<xarray.Dataset>
Dimensions:  (time: 1, x: 256, y_plus: 201, z: 256)
Coordinates:
  * x        (x) float64 0.0 0.04688 0.09375 0.1406 ... 11.81 11.86 11.91 11.95
  * y_plus   (y_plus) float64 790.0 790.0 789.8 789.6 ... 0.1949 0.04873 0.0
  * z        (z) float64 -3.0 -2.977 -2.953 -2.93 ... 2.906 2.93 2.953 2.977
  * time     (time) float64 2.632e+03
Data variables:
    u_vel    (time, x, y_plus, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    v_vel    (time, x, y_plus, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    w_vel    (time, x, y_plus, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr1      (time, x, y_plus, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.71   (time, x, y_plus, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.2    (time, x, y_plus, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.025  (time, x, y_plus, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>

First the computations are made for the velocities

In [14]:
ds.u_vel

<xarray.DataArray 'u_vel' (time: 1, x: 256, y_plus: 201, z: 256)>
dask.array<broadcast_to, shape=(1, 256, 201, 256), dtype=float64, chunksize=(1, 256, 201, 256), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 0.0 0.04688 0.09375 0.1406 ... 11.81 11.86 11.91 11.95
  * y_plus   (y_plus) float64 790.0 790.0 789.8 789.6 ... 0.1949 0.04873 0.0
  * z        (z) float64 -3.0 -2.977 -2.953 -2.93 ... 2.906 2.93 2.953 2.977
  * time     (time) float64 2.632e+03

Kører over alle de forskellige variable i datasettet. Skifter mellem skalaerer ved k=3

In [16]:
val_list=list(ds.keys())
mean=ds
mean=mean.drop(labels=val_list)
k=0
for val in val_list:
    mean_tem=ds[val]/u_tau
    mean_tem=xr.DataArray.mean(mean_tem,dim=('x','z'))
    rms_tem=(((ds[val]/u_tau)-mean_tem)**2)
    rms_tem=xr.ufuncs.sqrt(xr.DataArray.mean(rms_tem,dim=('x','z')))
    mean[val[0]+'_plusmean']=mean_tem
    mean[val[0]+'_plusRMS']=rms_tem
    if k>2: #So for the Pr numbers
        mean=mean.drop(labels=val[0]+'_plusmean')
        mean=mean.drop(labels=val[0]+'_plusRMS')
        pr=float(val[2:])
        theta_tau=(mean_tem.isel(y_plus=1)-mean_tem.isel(y_plus=0))/((mean_tem.y_plus[0]-mean_tem.y_plus[1])*pr)
        theta_plus=mean_tem/theta_tau
        rms_pr=rms_tem/theta_tau
        mean[val+'_plusmean']=theta_plus
        mean[val+'_plusRMS']=rms_pr
    k=k+1
    


In [ ]:
# %%time
# mean.to_zarr('data/interim/stats.nc')

In [ ]:
# %%time
# mean.to_netcdf('data/interim/stats.nc')

In [ ]:
a=mean.compute()
a.to_netcdf('data/interim/stats.nc')